In [13]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from tensorboardX import SummaryWriter
import argparse
from model import UNet          # UNet model
from dataset import *               # Dataset class & transform function
from utils import *                     # save & load function

In [14]:
parser = argparse.ArgumentParser(
    description="Train the Unet", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--lr", default=1e-3, type=float, dest="lr")
parser.add_argument("--batch_size", default=4, type=int, dest="batch_size")
parser.add_argument("--num_epoch", default=100, type=int, dest="num_epoch")
parser.add_argument("--data_dir", default=os.path.abspath('.')+"/data",
                    type=str, dest="data_dir")
parser.add_argument("--ckpt_dir", default=os.path.abspath('.')+"/checkpoint",
                    type=str, dest="ckpt_dir")
parser.add_argument("--log_dir", default=os.path.abspath('.') + "/log",
                    type=str, dest="log_dir")
parser.add_argument("--result_dir", default=os.path.abspath('.')+"/results", type=str, dest="result_dir")
parser.add_argument("--mode", default="train", type=str, dest="mode")
parser.add_argument("--train_continue", default="off", type=str, dest="train_continue")
args = parser.parse_args(args=[])


In [15]:
lr = args.lr
batch_size = args.batch_size
num_epoch = args.num_epoch
data_dir = args.data_dir
ckpt_dir = args.ckpt_dir
log_dir = args.log_dir
result_dir = args.result_dir
mode = args.mode
train_continue = args.train_continue

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the CPU


<img src="https://miro.medium.com/max/1200/1*qNdglJ1ORP3Gq77MmBLhHQ.png">

kernel_size(Convolution의 view, 2차원에서는 대부분 3x3 pixel), stride(이미지를 스캔할 때 kernel의 step size), bias는 CBR layer를 구축하는데 있어 값이 고정됨 >> 이 값들이 뭐하는 얘들이지?
각각의 함수가 의미하는 transforms

In [ ]:
if not os.path.exists(result_dir):
    os.makedirs(os.path.join(result_dir, "png"))
    os.makedirs(os.path.join(result_dir, "numpy"))

### train network

In [16]:
if mode == "train":
    transform = transforms.Compose([
        Normalization(mean=0.5, std=0.5),
        RandomFlip(),
        ToTensor(),
    ])
    dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=transform)
    loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

    dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=transform)
    loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers=0)
    # set variables
    num_data_train = len(dataset_train)
    num_data_val = len(dataset_val)

    num_batch_train = np.ceil(num_data_val / batch_size)
    num_batch_val = np.ceil(num_data_val / batch_size)
    
else:
    transform = transforms.Compose([
        Normalization(mean=0.5, std=0.5),
        ToTensor(),
    ])

    dataset_test = Dataset(data_dir=os.path.join(data_dir, 'test'), transform=transform)
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=0)
    # set variables
    num_data_test = len(dataset_test)

    num_batch_test = np.ceil(num_data_test / batch_size)


In [17]:
# build network
net = UNet().to(device)

In [18]:
# define loss function
fn_loss = nn.BCEWithLogitsLoss().to(device)

In [19]:
# set optimizer
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [20]:
# set variables
num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

num_batch_train = np.ceil(num_data_val / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

In [21]:
# sub functions

fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

In [22]:
# set SummaryWriter for Tensorboard
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

In [23]:
st_epoch =0
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

In [24]:
# train nework

st_epoch =0

if mode == "train":
    if train_continue == "on":
        net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)
    for epoch in range(st_epoch + 1, num_epoch +1):
        net.train()
        loss_arr = []
        for batch, data in enumerate(loader_train, 1):
            # forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)
            
            output = net(input)
            
            # backward pass
            
            optim.zero_grad()
            
            loss = fn_loss(output, label)
            loss.backward()
            
            optim.step()
            
            # calculate loss function
            loss_arr +=[loss.item()]
            
            print("Train: Epoch %04d / %04d | Batch %04d / %04d | Loss %.4f" %(epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr)))
            
            #save Tensorboard
            
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_class(output))
            
            writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
            writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        
        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
        
        with torch.no_grad():
            net.eval()
            loss_arr = []
            
            for batch, data in enumerate(loader_val, 1):
                
                # forward pass
                label = data['label'].to(device)
                input = data['input'].to(device)
                
                output = net(input)
                
                #calculate loss function
                loss = fn_loss(output, label)
                
                loss_arr += [loss.item()]
                
                print("Valid: Epoch %04d / %04d Batch %04d / %04d | Loss %.4f" %(epoch, num_epoch, batch, num_batch_val, np.mean(loss_arr)))
                
                # save Tensorboard 
                label = fn_tonumpy(label)
                input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
                output = fn_tonumpy(fn_class(output))
                
                writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
                writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
                writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
        
        save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)
    writer_train.close()
    writer_val.close()
     
else:
    # test nework
    net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)
    with torch.no_grad():
        net.eval()
        loss_arr = []
        
        for batch, data in enumerate(loader_test, 1):
            
            # forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)
            
            output = net(input)
            
            #calculate loss function
            loss = fn_loss(output, label)
            
            loss_arr += [loss.item()]
            
            print("Test: Batch %04d / %04d | Loss %.4f" %(batch, num_batch_test, np.mean(loss_arr)))
            
            # save Tensorboard 
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_class(output))
            
            for j in range(label.shape[0]):
                id = num_batch_test * (batch - 1) + j
                
                plt.imsave(os.path.join(result_dir, 'png', 'label_%04d.png' %id), label[j].squeeze(), cmap='gray')
                plt.imsave(os.path.join(result_dir, 'png', 'input_%04d.png' %id), input[j].squeeze(), cmap='gray')
                plt.imsave(os.path.join(result_dir, 'png', 'output_%04d.png' %id), output[j].squeeze(), cmap='gray')
                
                np.save(os.path.join(result_dir, 'numpy', 'label_%04d.npy' %id), label[j].squeeze())
                np.save(os.path.join(result_dir, 'numpy', 'input_%04d.npy' %id), input[j].squeeze())
                np.save(os.path.join(result_dir, 'numpy', 'output_%04d.npy' %id), output[j].squeeze())
                
    print("Average: Batch %04d / %04d | Loss %.4f" %(batch, num_batch_test, np.mean(loss_arr)))   

KeyboardInterrupt: 